Скрипт для компрессии и декомпрессии данных в игре Felix The Cat [NES]
--

Декомпрессия
---

Считывание данных из ROM, для удобства конвертируем их из символов в числа

In [1]:
from cadEditorPath import cadEditorDir
import os

romName = os.path.join(cadEditorDir, "Felix the Cat (U) [!].nes")
with open(romName, "rb") as f:
    d = f.read()
d = map(ord, d)
rom = d[:]

Вспомогательные функции

In [2]:
def readWord(d, addr):
    return d[addr+1]<<8 | d[addr]

def wordToBytes(word):
    return word&0xFF, word>>8


#прибавление адресов банков
def addBankOffset(addr, levelNo):
    if levelNo == 0:
        return addr + 0x10010
    elif levelNo == 1:
        return addr + 0xC010
    elif levelNo == 2:
        return addr + 0x8010
    else:
        return -1

#прибавление адресов банков
def addPtrToLinesPtr(addr, levelNo):
    return addBankOffset(addr, levelNo)

def addPtrToCompress(addr, levelNo):
    return addBankOffset(addr, levelNo)

#маскирование ненужных бит из адреса линий, которые кодируют информацию о списке объектов
def removeObjBits(addr):
    if addr < 0x8000:
        addr += 0x8000
    if addr > 0xC000:
        addr -= 0x4000
    return addr
    
#расчёт адресов линий
def addPtrToLines(addr, levelNo):
    addr = removeObjBits(addr)
    return addBankOffset(addr, levelNo)

Чтение поинтеров на массив адресов линий и словарь для распаковки из ROM

In [3]:
LEVEL_NO = 2

baseAddrs = [0x280, 0x2e0, 0x340]
baseAddr = baseAddrs[LEVEL_NO]
linesPtrsAddr = addPtrToLinesPtr(readWord(d,baseAddr-2), LEVEL_NO)
compressAddr = addPtrToCompress (readWord(d,baseAddr)  , LEVEL_NO)
    
print "Lines array addr   :", hex(linesPtrsAddr)
print "Compress dict addr :", hex(compressAddr)

Lines array addr   : 0x1201f
Compress dict addr : 0x11f4b


Чтение RLE-словаря для распаковки

In [4]:
def readCompress(d, addr):
    ans = []
    for c in xrange(128):
        ans.append((d[addr], d[addr+1]))
        addr += 2
    return ans

In [5]:
compressedArr = readCompress(d, compressAddr)
print "Compressed Array:", compressedArr

Compressed Array: [(24, 0), (11, 0), (7, 1), (10, 0), (6, 0), (4, 0), (2, 0), (3, 0), (2, 3), (13, 0), (12, 0), (9, 0), (6, 1), (7, 0), (8, 0), (3, 3), (4, 3), (2, 4), (4, 1), (5, 0), (2, 1), (8, 1), (14, 0), (5, 1), (3, 1), (11, 1), (9, 1), (3, 13), (3, 44), (4, 44), (16, 0), (15, 0), (18, 0), (19, 0), (20, 0), (17, 0), (4, 5), (2, 5), (5, 5), (3, 5), (2, 53), (2, 54), (6, 5), (2, 42), (2, 43), (163, 171), (164, 170), (164, 167), (169, 166), (166, 163), (163, 165), (169, 171), (163, 165), (170, 165), (165, 163), (172, 163), (34, 164), (163, 166), (162, 166), (169, 165), (164, 166), (163, 166), (169, 176), (178, 164), (35, 37), (169, 164), (168, 173), (162, 171), (177, 174), (164, 35), (165, 164), (172, 179), (164, 175), (36, 37), (40, 169), (174, 173), (175, 175), (173, 169), (174, 175), (175, 174), (169, 167), (172, 169), (171, 171), (169, 173), (168, 169), (175, 168), (172, 170), (169, 175), (169, 175), (170, 173), (175, 169), (173, 166), (175, 168), (166, 172), (169, 169), (175, 16

Хак для уровня 2 - чтение базовых линий, задающих повторяющуюся мозаику

In [6]:
#hardcode for pyramides levels (2-2 - 2-3)
def decompressLineForBackground(d, t, LINE_HEIGHT = 24):
    index = - (3 + t - 255)
    return decompressLine(d, 0x16627-index*24)

Распаковка одной линии

In [7]:
def decompressLine(d, addr, LINE_HEIGHT = 24):
    #print hex(addr)
    ans = []
    while len(ans) < LINE_HEIGHT:
        v = d[addr]
        #обычный тайл
        if v < 0x80:
            ans.append(v)
        #базовая линия (взять из линии-шаблона первые countFromBackground символов)
        elif v >= 0xFC:
            #print "Base line used"
            addr +=1
            backLine = decompressLineForBackground(d, v)
            countFromBackground = d[addr]
            ans.extend(backLine[:countFromBackground])
        #значение из RLE-словаря
        else:
            cv = v & 0x7F
            repeatCount, repeatTile = compressedArr[cv]
            ans.extend([repeatTile]*repeatCount)
        addr += 1
    return ans

Распаковка всех линий уровня

In [8]:
levelWidths = [256*3, 256*3, 256*3+128]
def decompressScreen():
    lines = []
    #actually, it's 3 separate configs for level x-1, x-2, x-3, but it has similar pointers and
    # lie together, so we simple read x3 times to get info of whole level
    WIDTH = levelWidths[LEVEL_NO]
    HEIGHT = 24
    curLinePtrsAddr = linesPtrsAddr
    for l in xrange(WIDTH):
        #print "CUR_LINE_PTR", hex(curLinePtrsAddr)
        lineAddr = addPtrToLines(readWord(d, curLinePtrsAddr), LEVEL_NO)
        #print "LINE_ADDR", hex(readWord(d, curLinePtrsAddr))
        vals = decompressLine(d, lineAddr, HEIGHT)
        #print vals
        if len(vals) != HEIGHT:
            print "Warning", len(vals)
            print vals
            vals = vals[:HEIGHT]
        ##assert(len(vals)== HEIGHT)
        lines.append(vals)
        curLinePtrsAddr += 2
    return lines

In [9]:
lines = decompressScreen()
screen = [item for sublist in lines for item in sublist]

In [10]:
dumpName = os.path.join(cadEditorDir, "settings_felix_the_cat/dump%d.bin"%(LEVEL_NO+1))
with open(dumpName, "wb") as f:
    v = "".join(map(chr, screen))
    f.write(v)

КОМПРЕССИЯ
---

In [11]:
from itertools import groupby

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

Считывание сохранённого дампа

In [12]:
dumpName = os.path.join(cadEditorDir, "settings_felix_the_cat/dump%d.bin")%(LEVEL_NO+1)
with open(dumpName, "rb") as f:
    d = f.read()
    
screen = map(ord, d)

Замена цепочки повторяющихся значений в линии на их индекс в RLE-словаре

In [13]:
def compressorReplace(lines, compressedPair, index):
    #replace with string version of lines for it's great replace method :)
    tc, ti = compressedPair
    findSeq = chr(ti)*tc
    ans = []
    findAtLeastOneReplace = False
    for line in lines:
        rline = "".join(chr(l) for l in line).replace(findSeq, chr(index | 0x80))
        rline = [ord(l) for l in rline]
        if line != rline:
            findAtLeastOneReplace = True
        #print "before:", compressedPair, line, index
        #print "after :", compressedPair, rline, index
        ans.append(rline)
    return ans, findAtLeastOneReplace

Построение потенциального RLE-словаря. Параметр forbiddenArr - список значений, которые не точно не будут использованы при построении реального словаря, используется для итеративного улучшения словаря.

In [14]:
def rebuildCompress(screen, forbiddenArr, maxCompressSize = 256, LINE_LEN=24):
    fullAns = [(0,0)]*maxCompressSize
    lines = list(chunks(screen, LINE_LEN))
    x = 0
    while x < maxCompressSize:
        ans = {}
        for line in lines:
            repeats = [list(g) for _, g in groupby(line)]
            repeats = [(g[0],len(g)) for g in repeats]
            for (tileNo,tileCount) in repeats:
                #for tc in xrange(tileCount, tileCount+1):
                for tc in xrange(2,tileCount+1):
                    compressPair = tileNo, tc
                    if compressPair in ans:
                        ans[compressPair] += 1
                    else:
                        ans[compressPair] = 1
        #рассчёт ценности замены - длины цепочки * частоты её встречаемости
        def calcPoints(v):
            (t,c), cnt = v
            return -(c-1)*cnt
    
        ans = sorted(ans.iteritems(), key = calcPoints)
        #filter and reverse values
        newAns = []
        for a in ans:
            if (a[0][0]) < 0x80:
                newAns.append((a[0][1],a[0][0]))
        
        newAns = filter(lambda v: v not in forbiddenArr, newAns)
        if len(newAns) == 0:
            break
            
        #HINT!!! if first results are similar in points, then we can use it's all
        ansValue = newAns[0][1]
        #newAns = filter(lambda x: x[1]==ansValue, newAns) #comment this for best results!
        #newAns = sorted(newAns, key = lambda x: -x[0])
        #print newAns
        similar, maxSimilar = 0, 256
        while len(newAns) > 0 and x < maxCompressSize and similar < maxSimilar:
            curAns = newAns[0]
            lines, findAtLeastOneReplace = compressorReplace(lines, curAns, x)
            fullAns[x] = curAns
            x += 1
            similar += 1
            newAns = newAns[1:]
    return fullAns

Простая версия построения словаря, без учёта перестройки линий и исключения неиспользуемых значений, для сравнения результатов

In [15]:
def rebuildCompressNoDinamic(screen, maxCompressSize = 64, LINE_LEN=24):
    ans = {}
    for line in chunks(screen, LINE_LEN):
        repeats = [list(g) for _, g in groupby(line)]
        repeats = [(g[0],len(g)) for g in repeats]
        for (tileNo,tileCount) in repeats:
            #for tc in xrange(tileCount, tileCount+1):
            for tc in xrange(2,tileCount+1):
                compressPair = tileNo, tc
                if compressPair in ans:
                    ans[compressPair] += 1
                else:
                    ans[compressPair] = 1
    #--
    def calcPoints(v):
        (t,c), cnt = v
        return -(c-1)*cnt
    
    ans = sorted(ans.iteritems(), key = calcPoints)
    #reverse values
    ans = map (lambda x: (x[0][1],x[0][0]), ans[:maxCompressSize])
    return ans

Построение RLE-словаря

In [16]:
#comment all this cell for no rebuild RLE-dictionary

#!!!
#forbidden = [(15, 15), (14, 15), (13, 15), (15, 5), (22, 124), (21, 124), (14, 5), (3, 108), (2, 112)]
#forbidden.extend([(12, 15), (20, 124), (13, 5), (19, 124), (2, 15), (3, 109)])
#forbidden.extend([(17, 15), (18, 124), (3, 110), (3, 80)])
#forbidden.extend([(11, 15), (2, 5), (2, 80), (0, 0)])
#

compressedArr = rebuildCompress(screen, [])
print "Compression array:", compressedArr

#compressedArr = rebuildCompressNoDinamic(screen)
#print compressedArr

Compression array: [(7, 0), (11, 0), (9, 0), (10, 0), (8, 0), (6, 0), (12, 0), (5, 0), (13, 0), (4, 0), (3, 0), (15, 0), (14, 0), (16, 0), (17, 0), (18, 0), (2, 0), (3, 3), (19, 0), (2, 3), (6, 1), (5, 1), (4, 1), (7, 1), (3, 1), (2, 1), (8, 1), (20, 0), (9, 1), (11, 1), (2, 4), (10, 1), (4, 3), (4, 5), (3, 5), (2, 5), (3, 44), (5, 5), (4, 44), (2, 44), (3, 13), (6, 5), (2, 54), (2, 13), (2, 42), (2, 43), (2, 53), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)

Запаковка строки. Выбирает самую длинную из возможных RLE-цепочек в словаре. Параметр compressedArrUsing возвращает количество использований каждого из значений словаря, что позволяет отследить неиспользуемые значения в словаре

In [17]:
def packRepeat(repeat, compressedArr, compressedArrUsing):
    tileNo, tileCount = repeat
    if tileCount <= 0:
        return []
    if tileCount == 1:
        return [tileNo]
    #---
    maxInd, maxCount = -1, -1
    for (ind,(c,t)) in enumerate(compressedArr):
        if t == tileNo and c <= tileCount:
            if c > maxCount:
                maxCount = c
                maxInd = ind
    if maxInd != -1:
        compressedArrUsing[maxInd] += 1
        return [0x80 | maxInd ] + packRepeat((tileNo, tileCount - maxCount), compressedArr, compressedArrUsing)
    #---
    return [tileNo] + packRepeat((tileNo, tileCount-1), compressedArr, compressedArrUsing)

def compressLine(l, compressedArr, compressedArrUsing):
    repeats = [list(g) for _, g in groupby(l)]
    repeats = [(g[0],len(g)) for g in repeats]
    ans = []
    for r in repeats:
        ans.extend(packRepeat(r, compressedArr, compressedArrUsing))
    return ans


Сжатие целого экрана

In [18]:
def compressScreen(screen, startDataAddr):
    LINE_LEN = 24
    curAddr = startDataAddr
    prevLinesAddrs = {}
    linesAddrs = []
    linesArray = []
    compressedArrUsing = [0] * len(compressedArr)
    for line in chunks(screen, LINE_LEN):
        cline = compressLine(line, compressedArr, compressedArrUsing)
        #print cline, ","
        clineTupple = tuple(cline)
        if clineTupple in prevLinesAddrs:
            linesAddrs.append(prevLinesAddrs[clineTupple])
            continue
        prevLinesAddrs[clineTupple] = curAddr
        linesAddrs.append(curAddr)
        linesArray.extend(cline)
        curAddr += len(cline)
    return linesAddrs, linesArray, compressedArrUsing

Последовательный подбор словаря и сжатие им с очисткой словаря

In [19]:
romPatchDataArrayAddr  = readWord(rom, linesPtrsAddr)

fullForbiddenArr = []

while True:
    linesAddrs, linesArray, compressedArrUsing = compressScreen(screen, romPatchDataArrayAddr)
    forbiddenArr = []
    print "Compressed size:", len(linesArray)
    #print compressedArrUsing
    for ca, car in zip(compressedArr, compressedArrUsing):
        if car == 0 and ca != (0,0):
            forbiddenArr.append(ca)
    #break #uncomment for no rebuild dict
    if forbiddenArr == []:
        break
    fullForbiddenArr.extend(forbiddenArr)
    compressedArr = rebuildCompress(screen, fullForbiddenArr)
compressedArr = filter (lambda v: v!=(0,0), compressedArr)
print "Final compressed arr (%d size):" % len(compressedArr), compressedArr

#for la in linesAddrs:
#    print hex(la)

Compressed size: 6251
Compressed size: 6251
Final compressed arr (44 size): [(7, 0), (11, 0), (9, 0), (10, 0), (8, 0), (6, 0), (12, 0), (5, 0), (13, 0), (4, 0), (3, 0), (15, 0), (14, 0), (16, 0), (17, 0), (18, 0), (2, 0), (3, 3), (19, 0), (2, 3), (6, 1), (5, 1), (4, 1), (7, 1), (3, 1), (2, 1), (8, 1), (20, 0), (9, 1), (11, 1), (2, 4), (4, 3), (4, 5), (3, 5), (2, 5), (3, 44), (5, 5), (4, 44), (3, 13), (6, 5), (2, 54), (2, 42), (2, 43), (2, 53)]


ПАТЧИНГ РОМА
--
Запаковка сжатых данных обратно в ROM-файл.
Возможны ошибки в случае, если сжатые данные занимают больше места, чем исходные

In [20]:
romName = os.path.join(cadEditorDir, "Felix the Cat (U) [!].nes")
romName2 = os.path.join(cadEditorDir, "Felix the Cat (U) [!]-2.nes")
with open(romName, "rb") as f:
    d = f.read()
    d = map(ord, d)

In [21]:
romPatchLinesArrayAddr = linesPtrsAddr
print "Lines   addr: ", hex(romPatchLinesArrayAddr)
romPatchCompressArrayAddr = compressAddr
print "RleDict addr: ", hex(romPatchCompressArrayAddr)
romPatchDataArrayAddr  = addBankOffset(readWord(d, linesPtrsAddr), LEVEL_NO)
print "Data    addr: ", hex(romPatchDataArrayAddr)


Lines   addr:  0x1201f
RleDict addr:  0x11f4b
Data    addr:  0x106e0


In [22]:
#convert pointer to bytes
dataLines = []
for addr in linesAddrs:
    dataLines.extend(wordToBytes(addr))

#write lines pointers
for x in xrange(len(dataLines)):
    #we must save some bits in hi-bytes addr, because they used to encoding objects
    curAddr = romPatchLinesArrayAddr+x
    if x % 2 == 1:
        v = d[romPatchLinesArrayAddr+x]
        v = (v & 0xC0) | (dataLines[x] & 0x3F)
        d[romPatchLinesArrayAddr+x] = v
    else:
        d[romPatchLinesArrayAddr+x] = dataLines[x]
#write lines data
print "Total data written:", len(linesArray)
#print linesArray
d[romPatchDataArrayAddr:romPatchDataArrayAddr+len(linesArray)] = linesArray
#write compress dict
compressedData = [y for x in compressedArr for y in x]
d[romPatchCompressArrayAddr:romPatchCompressArrayAddr+len(compressedData)] = compressedData
for i, dd in enumerate(d):
    if dd > 256:
        print hex(i), dd
with open(romName2, "wb") as f:
    f.write("".join(map(chr, d)))

Total data written: 6251


Для уровня 1:
<pre>
0. Размер сжатых данных со словарём, используемом в игре: 3740 байт
1. Размер данных со статическим составлением словаря:     3879 байт
(при этом до повтора целых линий - 6313 байт)
2. Размер данных с динамическим составлением словаря:     4695 байт
3. C поправкой на использование неск. возможных значений: 3838 байт
4. С циклическим исключением возможных ошибок:            3817 байт
5. С большим размером словаря:                            3740 байт
</pre>
Для уровня 3:
<pre>
Со словарём в игре : 6251 байт
С перестроенным    : 6251 байт
</pre>